In [ ]:
"""
使用预训练VGG16模型进行迁移学习

迁移学习(Transfer Learning)是计算机视觉领域的核心技术:
- 利用在大规模数据集(ImageNet)上预训练的模型
- 将学到的通用特征迁移到新任务
- 大幅减少训练时间和所需数据量

本notebook将演示特征提取方法:
1. 使用预训练VGG16作为特征提取器(冻结权重)
2. 仅训练顶层的全连接分类器
3. 对比从头训练的CNN模型

技术要点:
- VGG16在ImageNet上学到的特征具有很强的通用性
- 底层学习边缘/纹理,顶层学习复杂模式
- 特征提取速度快,训练成本低

作者: [Your Name]
日期: 2024-01
"""

import os
import warnings
from pathlib import Path

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 环境配置
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 设置随机种子确保可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")
print(f"GPU可用: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [ ]:
"""
数据路径配置和超参数设置
"""

# 数据路径(使用相对路径)
DATA_ROOT = Path("猫狗数据集/dataset")
TRAIN_DIR = DATA_ROOT / "train"
VALIDATION_DIR = DATA_ROOT / "validation"
TEST_DIR = DATA_ROOT / "test"

# 图像参数(必须与VGG16要求一致)
IMG_HEIGHT = 150
IMG_WIDTH = 150
IMG_CHANNELS = 3

# 训练参数
BATCH_SIZE = 20
EPOCHS = 2  # 测试用,生产环境建议30+
LEARNING_RATE = 2e-5  # 较小的学习率适合特征提取

# VGG16权重文件路径
# 可以设置为None让Keras自动下载,或指定本地文件
VGG_WEIGHTS_PATH = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

# 检查权重文件是否存在
if VGG_WEIGHTS_PATH and Path(VGG_WEIGHTS_PATH).exists():
    print(f"✓ 使用本地VGG16权重: {VGG_WEIGHTS_PATH}")
    weights_source = VGG_WEIGHTS_PATH
elif VGG_WEIGHTS_PATH:
    print(f"⚠️  本地权重文件不存在: {VGG_WEIGHTS_PATH}")
    print("将使用在线下载的权重")
    weights_source = 'imagenet'
else:
    print("使用在线下载的ImageNet权重")
    weights_source = 'imagenet'

print("\n" + "=" * 60)
print("配置信息")
print("=" * 60)
print(f"图像尺寸: {IMG_HEIGHT}x{IMG_WIDTH}x{IMG_CHANNELS}")
print(f"批次大小: {BATCH_SIZE}")
print(f"训练轮数: {EPOCHS} (注意: 测试配置)")
print(f"学习率: {LEARNING_RATE}")
print("=" * 60)

In [ ]:
"""
加载预训练的VGG16卷积基

VGG16架构:
- 13个卷积层 + 3个全连接层
- 5个池化层将图像尺寸递减
- 总参数约1.38亿个

参数说明:
- weights: 使用ImageNet预训练权重
- include_top=False: 不包括顶层全连接层(我们将自定义分类器)
- input_shape: 输入图像尺寸

为什么不包括顶层?
- ImageNet有1000个类别,我们只需要2个(猫/狗)
- 自定义分类器更灵活,可根据任务调整
"""

# 加载VGG16卷积基(不包括全连接层)
conv_base = VGG16(
    weights=weights_source,
    include_top=False,  # 不包括顶层分类器
    input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
)

print("=" * 60)
print("VGG16卷积基架构")
print("=" * 60)
conv_base.summary()
print("=" * 60)

# 分析模型参数
total_params = conv_base.count_params()
print(f"\n卷积基参数量: {total_params:,} ({total_params/1e6:.2f}M)")
print(f"输出形状: {conv_base.output_shape}")
print("=" * 60)

In [ ]:
"""
使用VGG16提取图像特征

特征提取流程:
1. 使用VGG16将图像转换为特征向量
2. 将提取的特征保存到内存
3. 后续用这些特征训练轻量级分类器

优势:
- 特征提取只需一次,节省训练时间
- 可以使用更大的批次训练分类器
- 适合计算资源受限的场景

数据集样本数量配置
"""

# 数据集样本数量(根据实际数据调整)
TRAIN_SAMPLES = 10000   # 训练集样本数
VAL_SAMPLES = 2500      # 验证集样本数
TEST_SAMPLES = 6200     # 测试集样本数

print("=" * 60)
print("数据集规模")
print("=" * 60)
print(f"训练样本数: {TRAIN_SAMPLES}")
print(f"验证样本数: {VAL_SAMPLES}")
print(f"测试样本数: {TEST_SAMPLES}")
print(f"总样本数: {TRAIN_SAMPLES + VAL_SAMPLES + TEST_SAMPLES}")
print("=" * 60)

In [ ]:
"""
定义特征提取函数并提取所有数据集的特征

关键技术点:
1. 使用ImageDataGenerator进行图像归一化
2. 批量处理图像提高效率
3. 使用conv_base.predict()提取特征
4. 同时保存特征和标签

注意:
- 特征提取阶段不使用数据增强
- VGG16输出的特征形状为(4, 4, 512)
- 后续需要展平为1维向量用于全连接层
"""

def extract_features(directory, sample_count, batch_size=20):
    """
    使用VGG16卷积基提取图像特征
    
    Args:
        directory: 图像目录路径
        sample_count: 要提取的样本数量
        batch_size: 批次大小
        
    Returns:
        features: 提取的特征数组, shape=(sample_count, 4, 4, 512)
        labels: 对应的标签数组, shape=(sample_count,)
    """
    # 初始化特征和标签数组
    features = np.zeros(shape=(sample_count, 4, 4, 512), dtype=np.float32)
    labels = np.zeros(shape=(sample_count), dtype=np.float32)
    
    # 创建数据生成器(仅归一化)
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        directory,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False  # 保持顺序以确保标签对应
    )
    
    # 批量提取特征
    print(f"从 {Path(directory).name} 提取特征...")
    i = 0
    for inputs_batch, labels_batch in generator:
        # 使用VGG16提取特征
        features_batch = conv_base.predict(inputs_batch, verbose=0)
        
        # 计算当前批次的实际大小和索引范围
        batch_size_actual = inputs_batch.shape[0]
        start_idx = i * batch_size
        end_idx = start_idx + batch_size_actual
        
        # 保存特征和标签
        features[start_idx:end_idx] = features_batch
        labels[start_idx:end_idx] = labels_batch
        
        i += 1
        
        # 显示进度
        if i % 50 == 0:
            print(f"  已处理: {min(end_idx, sample_count)}/{sample_count} 样本")
        
        # 达到目标样本数则停止
        if end_idx >= sample_count:
            break
    
    print(f"✓ 完成! 提取了 {sample_count} 个样本的特征\n")
    return features, labels

# 提取所有数据集的特征
print("=" * 60)
print("开始提取特征")
print("=" * 60)
print("这可能需要几分钟时间,请耐心等待...\n")

train_features, train_labels = extract_features(str(TRAIN_DIR), TRAIN_SAMPLES, BATCH_SIZE)
validation_features, validation_labels = extract_features(str(VALIDATION_DIR), VAL_SAMPLES, BATCH_SIZE)
test_features, test_labels = extract_features(str(TEST_DIR), TEST_SAMPLES, BATCH_SIZE)

print("=" * 60)
print("特征提取完成")
print("=" * 60)
print(f"训练特征形状: {train_features.shape}")
print(f"验证特征形状: {validation_features.shape}")
print(f"测试特征形状: {test_features.shape}")
print("=" * 60)

In [ ]:
"""
展平特征向量

将4D特征张量展平为2D:
- 输入形状: (samples, 4, 4, 512)
- 输出形状: (samples, 8192)

为什么需要展平?
- 全连接层只接受1D输入
- 4*4*512 = 8192维特征向量
"""

# 展平特征
train_features_flat = np.reshape(train_features, (TRAIN_SAMPLES, 4 * 4 * 512))
validation_features_flat = np.reshape(validation_features, (VAL_SAMPLES, 4 * 4 * 512))
test_features_flat = np.reshape(test_features, (TEST_SAMPLES, 4 * 4 * 512))

print("=" * 60)
print("特征展平")
print("=" * 60)
print(f"训练特征: {train_features.shape} → {train_features_flat.shape}")
print(f"验证特征: {validation_features.shape} → {validation_features_flat.shape}")
print(f"测试特征: {test_features.shape} → {test_features_flat.shape}")
print(f"\n每个样本的特征维度: {train_features_flat.shape[1]:,}")
print("=" * 60)

In [ ]:
"""
构建并训练全连接分类器

网络结构:
- 输入: 8192维特征向量(来自VGG16)
- 隐藏层: 256个神经元 + ReLU激活
- Dropout: 0.5比例(防止过拟合)
- 输出层: 1个神经元 + Sigmoid激活(二分类)

训练策略:
- 使用较小学习率(2e-5)
- 添加EarlyStopping避免过拟合
- 监控验证损失,patience=3
"""

from tensorflow.keras.callbacks import EarlyStopping

# 构建分类器模型
classifier = models.Sequential(name='VGG16_Classifier')
classifier.add(layers.Dense(
    256, 
    activation='relu', 
    input_dim=4*4*512,  # 输入维度必须与特征维度匹配
    name='fc1'
))
classifier.add(layers.Dropout(0.5, name='dropout'))
classifier.add(layers.Dense(1, activation='sigmoid', name='output'))

# 编译模型
classifier.compile(
    optimizer=optimizers.RMSprop(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("=" * 60)
print("分类器架构")
print("=" * 60)
classifier.summary()
print("=" * 60)

# 配置早停回调
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,  # 恢复最佳权重
    verbose=1
)

print("\n" + "=" * 60)
print("开始训练分类器")
print("=" * 60)
print(f"训练样本: {len(train_features_flat)}")
print(f"验证样本: {len(validation_features_flat)}")
print(f"批次大小: 32")
print(f"最大轮数: {EPOCHS}")
print(f"早停策略: 验证损失3轮无改善则停止")
print("=" * 60 + "\n")

# 训练分类器
history = classifier.fit(
    train_features_flat, 
    train_labels,
    epochs=EPOCHS,
    batch_size=32,
    validation_data=(validation_features_flat, validation_labels),
    callbacks=[early_stopping],
    verbose=1
)

print("\n" + "=" * 60)
print("训练完成")
print("=" * 60)

# 显示最终结果
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print(f"\n最终结果:")
print(f"  训练准确率: {final_train_acc:.4f}")
print(f"  验证准确率: {final_val_acc:.4f}")
print(f"  训练损失: {final_train_loss:.4f}")
print(f"  验证损失: {final_val_loss:.4f}")
print("=" * 60)

In [ ]:
"""
可视化训练过程

展示准确率和损失的变化趋势,评估模型性能
"""

import matplotlib.pyplot as plt

# 提取训练历史
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(acc) + 1)

# 创建图表
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 绘制准确率
ax1.plot(epochs_range, acc, 'bo-', label='训练准确率', linewidth=2, markersize=8)
ax1.plot(epochs_range, val_acc, 'rs-', label='验证准确率', linewidth=2, markersize=8)
ax1.set_title('训练和验证准确率 (VGG16特征提取)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('准确率', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# 绘制损失
ax2.plot(epochs_range, loss, 'bo-', label='训练损失', linewidth=2, markersize=8)
ax2.plot(epochs_range, val_loss, 'rs-', label='验证损失', linewidth=2, markersize=8)
ax2.set_title('训练和验证损失 (VGG16特征提取)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('损失', fontsize=12)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 性能分析
print("\n" + "=" * 60)
print("性能分析")
print("=" * 60)
print(f"验证准确率: {val_acc[-1]:.4f}")
print(f"训练轮数: {len(acc)}")

if val_acc[-1] > 0.9:
    print("\n✓ 优秀! 使用预训练特征获得了很好的效果")
    print("  VGG16学到的特征在新任务上泛化良好")
elif val_acc[-1] > 0.85:
    print("\n✓ 良好! 特征提取方法有效")
else:
    print("\n⚠️  性能有提升空间,可以考虑:")
    print("  - 增加训练轮数")
    print("  - 调整分类器架构")
    print("  - 尝试模型微调(fine-tuning)")
print("=" * 60)

In [ ]:
"""
方法2: 端到端训练(冻结卷积基)

与特征提取不同,这种方法:
1. 将VGG16卷积基作为模型的一部分
2. 冻结卷积基权重(不训练)
3. 仅训练顶层分类器
4. 使用数据生成器直接训练

优势:
- 可以使用数据增强
- 内存占用更小(不需要保存所有特征)
- 更适合大数据集

对比特征提取法:
- 特征提取: 先提取特征→再训练分类器(两阶段)
- 端到端: 直接在原始图像上训练(一阶段)
"""

# 构建端到端模型
model_frozen = models.Sequential(name='VGG16_Frozen_E2E')
model_frozen.add(conv_base)  # 添加VGG16卷积基
model_frozen.add(layers.Flatten(name='flatten'))
model_frozen.add(layers.Dense(256, activation='relu', name='fc1'))
model_frozen.add(layers.Dropout(0.5, name='dropout'))
model_frozen.add(layers.Dense(1, activation='sigmoid', name='output'))

print("=" * 60)
print("端到端模型架构 (冻结卷积基)")
print("=" * 60)
model_frozen.summary()
print("=" * 60)

In [ ]:
"""
冻结卷积基权重

为什么要冻结?
- VGG16已在ImageNet上充分训练
- 底层特征(边缘、纹理)具有通用性
- 仅训练顶层分类器速度更快
- 避免破坏预训练的特征

如何冻结?
- 设置conv_base.trainable = False
- 冻结后的层不参与反向传播
- 权重在训练过程中保持不变
"""

# 冻结卷积基的所有层
conv_base.trainable = False

# 验证冻结状态
print("=" * 60)
print("模型参数统计")
print("=" * 60)
trainable_count = sum([np.prod(w.shape) for w in model_frozen.trainable_weights])
non_trainable_count = sum([np.prod(w.shape) for w in model_frozen.non_trainable_weights])
total_count = trainable_count + non_trainable_count

print(f"可训练参数: {trainable_count:,} ({trainable_count/1e6:.2f}M)")
print(f"冻结参数: {non_trainable_count:,} ({non_trainable_count/1e6:.2f}M)")
print(f"总参数: {total_count:,} ({total_count/1e6:.2f}M)")
print(f"\n参数减少比例: {(non_trainable_count/total_count)*100:.1f}% 的参数被冻结")
print("=" * 60)

In [ ]:
"""
使用数据增强进行端到端训练

训练配置:
- 训练集使用数据增强
- 验证集仅归一化(不增强)
- 使用较小学习率保护预训练权重
- 添加EarlyStopping防止过拟合
"""

# 创建数据生成器
train_datagen_e2e = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen_e2e = ImageDataGenerator(rescale=1./255)

# 创建数据流
train_generator_e2e = train_datagen_e2e.flow_from_directory(
    str(TRAIN_DIR),
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    seed=RANDOM_SEED
)

validation_generator_e2e = val_datagen_e2e.flow_from_directory(
    str(VALIDATION_DIR),
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# 编译模型
model_frozen.compile(
    optimizer=optimizers.RMSprop(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 计算训练步数
steps_per_epoch_e2e = train_generator_e2e.samples // 32
validation_steps_e2e = validation_generator_e2e.samples // 32

# 配置回调
early_stopping_e2e = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

print("=" * 60)
print("开始端到端训练 (冻结VGG16 + 数据增强)")
print("=" * 60)
print(f"训练样本: {train_generator_e2e.samples}")
print(f"验证样本: {validation_generator_e2e.samples}")
print(f"批次大小: 32")
print(f"最大轮数: {EPOCHS}")
print(f"数据增强: 已启用")
print("=" * 60 + "\n")

# 训练模型
history_e2e = model_frozen.fit(
    train_generator_e2e,
    steps_per_epoch=steps_per_epoch_e2e,
    epochs=EPOCHS,
    validation_data=validation_generator_e2e,
    validation_steps=validation_steps_e2e,
    callbacks=[early_stopping_e2e],
    verbose=1
)

print("\n" + "=" * 60)
print("端到端训练完成")
print("=" * 60)

# 显示结果
final_train_acc_e2e = history_e2e.history['accuracy'][-1]
final_val_acc_e2e = history_e2e.history['val_accuracy'][-1]

print(f"\n最终结果:")
print(f"  训练准确率: {final_train_acc_e2e:.4f}")
print(f"  验证准确率: {final_val_acc_e2e:.4f}")
print(f"\n对比特征提取法:")
print(f"  特征提取验证准确率: {final_val_acc:.4f}")
print(f"  端到端验证准确率: {final_val_acc_e2e:.4f}")
print(f"  差异: {final_val_acc_e2e - final_val_acc:+.4f}")
print("=" * 60)

In [ ]:
"""
可视化端到端训练结果
"""

import matplotlib.pyplot as plt

# 提取训练历史
acc_e2e = history_e2e.history['accuracy']
val_acc_e2e = history_e2e.history['val_accuracy']
loss_e2e = history_e2e.history['loss']
val_loss_e2e = history_e2e.history['val_loss']
epochs_range_e2e = range(1, len(acc_e2e) + 1)

# 创建图表
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 准确率曲线
ax1.plot(epochs_range_e2e, acc_e2e, 'bo-', label='训练准确率', linewidth=2, markersize=8)
ax1.plot(epochs_range_e2e, val_acc_e2e, 'rs-', label='验证准确率', linewidth=2, markersize=8)
ax1.set_title('端到端训练: 准确率', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('准确率', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# 损失曲线
ax2.plot(epochs_range_e2e, loss_e2e, 'bo-', label='训练损失', linewidth=2, markersize=8)
ax2.plot(epochs_range_e2e, val_loss_e2e, 'rs-', label='验证损失', linewidth=2, markersize=8)
ax2.set_title('端到端训练: 损失', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('损失', fontsize=12)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("总结")
print("=" * 60)
print("\n特征提取 vs 端到端训练:")
print(f"  方法1 (特征提取): 验证准确率 = {final_val_acc:.4f}")
print(f"  方法2 (端到端): 验证准确率 = {final_val_acc_e2e:.4f}")
print("\n两种方法各有优势:")
print("  特征提取:")
print("    ✓ 训练速度快")
print("    ✓ 内存需求小")
print("    ✗ 无法使用数据增强")
print("\n  端到端:")
print("    ✓ 可以使用数据增强")
print("    ✓ 性能通常更好")
print("    ✗ 训练速度较慢")
print("=" * 60)

In [ ]:
"""
总结与展望

本notebook演示了迁移学习的特征提取方法:
✓ 使用预训练VGG16提取图像特征
✓ 对比两种训练方式(特征提取 vs 端到端)
✓ 理解冻结权重的重要性
✓ 观察迁移学习的性能优势

关键收获:
1. 迁移学习显著减少训练时间和数据需求
2. 预训练特征具有很强的通用性
3. 特征提取适合快速原型和小数据集
4. 端到端训练性能更好但成本更高

下一步:
- 模型微调(Fine-tuning): 解冻部分卷积层进一步优化
- 尝试其他预训练模型(ResNet, EfficientNet等)
- 在测试集上评估最终性能
- 优化超参数(学习率, Dropout率等)
"""

print("=" * 60)
print("Notebook执行完成")
print("=" * 60)
print("\n迁移学习优势总结:")
print("  ✓ 大幅减少训练时间")
print("  ✓ 在小数据集上也能获得好效果")
print("  ✓ 利用ImageNet上学到的通用特征")
print("\n推荐阅读:")
print("  - 使用模型微调分类猫狗.ipynb")
print("  - 了解如何解冻层进行Fine-tuning")
print("=" * 60)